In [3]:
from atj_algotrading.backtester import get_ohlc_history, create_price_fig, Backtester, evaluate_backtest
from datetime import datetime, timedelta
import MetaTrader5 as mt5
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd

In [6]:
mt5.initialize()

symbol = 'US500'
start_dt = datetime.now() - timedelta(days=365 * 3)
end_dt = datetime.now()

# get historical data
mt5.initialize()
ohlc_d1 = get_ohlc_history(symbol, mt5.TIMEFRAME_D1, start_dt, end_dt)
ohlc_d1['date'] = ohlc_d1['time'].dt.date
ohlc_d1 = ohlc_d1.rename(columns={'high': 'daily_high', 'low': 'daily_low'})

display(ohlc_d1)

,time,open,daily_high,daily_low,close,date
0,2021-10-13,4337.19,4374.22,4329.20,4364.63,2021-10-13
1,2021-10-14,4363.57,4444.89,4363.57,4444.14,2021-10-14
2,2021-10-15,4441.93,4475.84,4435.19,4472.73,2021-10-15
3,2021-10-18,4473.87,4488.86,4444.87,4484.36,2021-10-18
4,2021-10-19,4484.59,4526.17,4481.09,4524.05,2021-10-19
...,...,...,...,...,...,...
771,2024-10-07,5750.41,5758.41,5686.48,5701.40,2024-10-07
772,2024-10-08,5703.69,5757.35,5676.79,5749.12,2024-10-08
773,2024-10-09,5749.53,5797.05,5732.22,5787.02,2024-10-09
774,2024-10-10,5786.55,5795.18,5763.91,5782.79,2024-10-10


In [20]:
mt5.initialize()
ohlc_h1 = get_ohlc_history(symbol, mt5.TIMEFRAME_H1, start_dt, end_dt)
ohlc_h1['date'] = ohlc_h1['time'].dt.date

ohlc_max_values = ohlc_h1.sort_values(['date', 'high'], ascending=[True, False])[['time', 'date', 'high']]
ohlc_max_values2 = ohlc_max_values.groupby('date').first()
ohlc_max_values2['type'] = 'high'
ohlc_max_values2 = ohlc_max_values2.rename(columns={'high': 'value'})

ohlc_min_values = ohlc_h1.sort_values(['date', 'low'], ascending=[True, True])[['time', 'date', 'low']]
ohlc_min_values2 = ohlc_min_values.groupby('date').first()
ohlc_min_values2['type'] = 'low'
ohlc_min_values2 = ohlc_min_values2.rename(columns={'low': 'value'})

display(ohlc_max_values2)
display(ohlc_min_values2)

min_max_df = pd.concat([ohlc_max_values2, ohlc_min_values2])
min_max_df = min_max_df.sort_values('time')
display(min_max_df)

px.line(min_max_df, x='time', y='value')

,time,value,type
date,,,
2021-10-12,2021-10-12 15:00:00,4374.48,high
2021-10-13,2021-10-13 21:00:00,4374.22,high
2021-10-14,2021-10-14 23:00:00,4444.89,high
2021-10-15,2021-10-15 21:00:00,4475.84,high
2021-10-18,2021-10-18 22:00:00,4488.86,high
...,...,...,...
2024-10-07,2024-10-07 01:00:00,5758.41,high
2024-10-08,2024-10-08 22:00:00,5757.35,high
2024-10-09,2024-10-09 22:00:00,5797.05,high


,time,value,type
date,,,
2021-10-12,2021-10-12 23:00:00,4336.22,low
2021-10-13,2021-10-13 17:00:00,4329.20,low
2021-10-14,2021-10-14 01:00:00,4363.57,low
2021-10-15,2021-10-15 04:00:00,4435.19,low
2021-10-18,2021-10-18 15:00:00,4444.87,low
...,...,...,...
2024-10-07,2024-10-07 21:00:00,5686.48,low
2024-10-08,2024-10-08 01:00:00,5676.79,low
2024-10-09,2024-10-09 10:00:00,5732.22,low


,time,value,type
date,,,
2021-10-12,2021-10-12 15:00:00,4374.48,high
2021-10-12,2021-10-12 23:00:00,4336.22,low
2021-10-13,2021-10-13 17:00:00,4329.20,low
2021-10-13,2021-10-13 21:00:00,4374.22,high
2021-10-14,2021-10-14 01:00:00,4363.57,low
...,...,...,...
2024-10-09,2024-10-09 22:00:00,5797.05,high
2024-10-10,2024-10-10 19:00:00,5795.18,high
2024-10-10,2024-10-10 21:00:00,5763.91,low


In [52]:
min_max_df2 = min_max_df.copy()

min_max_df2['prev_type'] = min_max_df2['type'].shift(1)
min_max_df2['prev_value'] = min_max_df2['value'].shift(1)
min_max_df2['prev_time'] = min_max_df2['time'].shift(1)
min_max_df2['same_type_in_a_row'] = min_max_df2.apply(lambda x: 1 if x['type'] == x['prev_type'] else 0, axis=1)

def choose_min_max_value(x):
    if x['same_type_in_a_row'] == 1 and x['type'] == 'low':
        if x['value'] < x['prev_value']:
            return x['value'], x['time']
        else:
            return x['prev_value'], x['prev_time']
        
    elif x['same_type_in_a_row'] == 1 and x['type'] == 'high':
        if x['value'] > x['prev_value']:
            return x['value'], x['time']
        else:
            return x['prev_value'], x['prev_time']
    
min_max_df2['new_data'] = min_max_df2.apply(choose_min_max_value, axis=1)
min_max_df2['new_data'] = min_max_df2['new_data'].shift(-1)

def overwrite_value(x):
    if x['new_data']:
        return x['new_data'][0]
    else:
        return x['value']
    
def overwrite_time(x):
    if x['new_data']:
        return x['new_data'][1]
    else:
        return x['time']

min_max_df2['time'] = min_max_df2.apply(overwrite_time, axis=1)
min_max_df2['value'] = min_max_df2.apply(overwrite_value, axis=1)

min_max_df2 = min_max_df2[min_max_df2['same_type_in_a_row'] == 0]
min_max_df2

,time,value,type,prev_type,prev_value,prev_time,same_type_in_a_row,new_data
date,,,,,,,,
2021-10-12,2021-10-12 15:00:00,4374.48,high,None,NaN,NaT,0,None
2021-10-12,2021-10-13 17:00:00,4329.20,low,high,4374.48,2021-10-12 15:00:00,0,"(4329.2, 2021-10-13 17:00:00)"
2021-10-13,2021-10-13 21:00:00,4374.22,high,low,4329.20,2021-10-13 17:00:00,0,None
2021-10-14,2021-10-14 01:00:00,4363.57,low,high,4374.22,2021-10-13 21:00:00,0,None
2021-10-14,2021-10-14 23:00:00,4444.89,high,low,4363.57,2021-10-14 01:00:00,0,None
...,...,...,...,...,...,...,...,...
2024-10-09,2024-10-09 10:00:00,5732.22,low,high,5757.35,2024-10-08 22:00:00,0,None
2024-10-09,2024-10-09 22:00:00,5797.05,high,low,5732.22,2024-10-09 10:00:00,0,"(5797.05, 2024-10-09 22:00:00)"
2024-10-10,2024-10-10 21:00:00,5763.91,low,high,5795.18,2024-10-10 19:00:00,0,None


In [54]:
fig = create_price_fig(ohlc_h1)
fig.add_trace(go.Scatter(x=min_max_df2['time'], y=min_max_df2['value'], mode="lines"))

fig.show()

In [57]:
px.line(min_max_df2, x=min_max_df2.reset_index().index, y='value')